# Equilibrium Solver #
The Equilibrium solver is designed to take a reaction network as input and solve for a correct equilibrium solution. It does this by writing the system as a system of equations. Solving these equations can give the expected concentrations of all species at Equilibrium.

It is useful to have this capability because we want to compare the results of our simulations to equilibrium. This allows for the detection of traps and other interesting kinetic effects. 

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../")

In [2]:
from steric_free_simulator import VectorizedRxnNet, ReactionNetwork, EquilibriumSolver

EnergyExplorer Module is not available. Check Rosetta installation. <ipykernel.iostream.OutStream object at 0x7f1002a4bcc0>


As usual, we can start by loading up the reaction network

In [3]:
base_input = '../input_files/trimer.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()

['A']
['B']
['C']
-----
{'A'}
{'A'}
set()
-----
{'A'}
{'B'}
{'A'}
Connected Nodes:  ['A', 'B']
Connected Edges:  [('A', 'B')]
New node added
[0, 1, 2, 3]
-----
{'A'}
{'C'}
{'A'}
Connected Nodes:  ['A', 'C']
Connected Edges:  [('A', 'C')]
New node added
[0, 1, 2, 3, 4]
-----
{'B'}
{'A'}
{'B'}
Connected Nodes:  ['B', 'A']
Connected Edges:  [('B', 'A')]
-----
{'B'}
{'B'}
set()
-----
{'B'}
{'C'}
{'B'}
Connected Nodes:  ['B', 'C']
Connected Edges:  [('B', 'C')]
New node added
[0, 1, 2, 3, 4, 5]
-----
{'B'}
{'A', 'B'}
set()
-----
{'B'}
{'C', 'A'}
{'B'}
Connected Nodes:  ['B', 'A', 'C']
Connected Edges:  [('B', 'A'), ('A', 'C')]
Connected Nodes:  ['B', 'A', 'C']
Connected Edges:  [('B', 'A'), ('B', 'C'), ('A', 'C')]
New node added
[0, 1, 2, 3, 4, 5, 6]
-----
{'C'}
{'A'}
{'C'}
Connected Nodes:  ['C', 'A']
Connected Edges:  [('C', 'A')]
-----
{'C'}
{'B'}
{'C'}
Connected Nodes:  ['C', 'B']
Connected Edges:  [('C', 'B')]
-----
{'C'}
{'C'}
set()
-----
{'C'}
{'A', 'B'}
{'C'}
Connected Nodes:  ['C',

A minor annoyance is that the reaction network need the association constants to be resolved, which normally happens at simulation time. We can work around this by generated the vectorized network then writing it back to the normal reaction network.

In [4]:
vec_rn = VectorizedRxnNet(rn)
vec_rn.update_reaction_net(rn)

Now we will initialize a equilibrium solver object on the reaction network. The constructor will convert the network to a list of polynomial equations and constraints defining the system at equilibrium. For simlisty all interactions are written as there own equation, all simplification is left to the sympy engine. 

Sympy is an open-source module that allows python programs to do symbollic math, similar to closed-source tools like Mathematica. 

In [5]:
eq = EquilibriumSolver(rn)

In order to solve the system of equations, we call EquilbriumSolver's `solve` method. Internally, this call `sympy.nsolve` which uses a variety of numeric methods to find a solution. Since the solver is sensitive to initialization, if a solution is not found a random restart is preformed up to a set number of times.

The result will be a vector with copy numbers for each species. Since this system is a trimer, the first three indices  are the equilibrium monomer subunit concentrations, and as always the last index in the vector is the equilibrium concentration of the complete complex. The other values are equilibrium concentrations of various intermediates. 

In [6]:
sol = eq.solve()
sol

Matrix([
[0.7380988],
[ 1.618495],
[ 1.447663],
[ 1.488828],
[ 1.659661],
[0.7792647],
[ 1.113412]])

We can now easily calculate the expected equilibrium complet complex yield using our definition of yield.

In [7]:
print("Equilibrium expected yield: ", 100 * sol[-1] / min(vec_rn.initial_copies[:vec_rn.num_monomers]), '%')

Equilibrium expected yield:  22.2682336807251 %
